In [1]:
using Revise

using RigidBodyDynamics
using RigidBodyDynamics: Bounds

using DrakeVisualizer
using Plots
using RigidBodyTreeInspector

using BilevelTrajOpt
gr(size=(500,500), html_output_format=:png);

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/8c8a043c2f00369f3e7ae550277d2a54e59a34ad-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/8c8a043c2f00369f3e7ae550277d2a54e59a34ad-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [2]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)
body = findbody(mechanism, "ball")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100)
μ = 0.5
motion_type = :xyz
point = Point3D(default_frame(findbody(mechanism, "floor")), [0.,0.,0.])
normal = FreeVector3D(default_frame(findbody(mechanism, "floor")), [0.,0.,1.])
hs = HalfSpace(point, normal)
floor = Obstacle(hs, μ, motion_type)
obstacles = [floor]
env = parse_contacts(mechanism, urdf, obstacles)
x0 = MechanismState(mechanism)
Δt = 0.005;

In [3]:
q0 = [1., 0., 0., 0., 0., 0., 0.025]
v0 = [0., 0., 0., 0.5, 0. , 0.]
N = 5

set_configuration!(x0, q0)
set_velocity!(x0, v0)
setdirty!(x0)

ctrl! = (u,t,x) -> u[:] = 0.

(::#1) (generic function with 1 method)

In [7]:
println("Implicit Contacts")
traj_imp = BilevelTrajOpt.simulate(x0,env,Δt,N,ctrl!,implicit_contact=true);

Implicit Contacts


LoadError: [91mInterruptException:[39m

In [6]:
println("Explicit Contacts")
traj_exp = BilevelTrajOpt.simulate(x0,env,Δt,N,ctrl!,implicit_contact=false);

Explicit Contacts


elapsed time: 1.307128095 seconds
elapsed time: 1.221925384 seconds
elapsed time: 1.168588222 seconds
elapsed time: 1.197729165 seconds
elapsed time: 1.224758226 seconds
elapsed time: 1.187326412 seconds
elapsed time: 1.203459047 seconds
elapsed time: 1.20054364 seconds
elapsed time: 1.217445082 seconds
elapsed time: 1.192321253 seconds
elapsed time: 1.198688902 seconds
elapsed time: 1.176230107 seconds
elapsed time: 1.192592379 seconds
elapsed time: 1.193210031 seconds
elapsed time: 1.216113083 seconds
elapsed time: 1.199898138 seconds
elapsed time: 1.188218401 seconds
elapsed time: 1.17332226 seconds
elapsed time: 1.193557257 seconds
elapsed time: 1.179785281 seconds
elapsed time: 1.207773823 seconds
elapsed time: 1.178412624 seconds
Restoration_Failed
elapsed time: 1.207711215 seconds
elapsed time: 1.201000948 seconds
elapsed time: 1.190925825 seconds
elapsed time: 1.205799658 seconds
elapsed time: 1.18260437 seconds
elapsed time: 1.19177939 seconds
elapsed time: 1.189853653 seconds

LoadError: [91mInterruptException:[39m

In [ ]:
ts = cumsum([Δt for i in 1:N]);
j = 7
plot(traj_exp[j,:],label="explicit")
# plot!(traj_imp[j,:],label="implicit")

In [ ]:
traj_imp

In [ ]:
maximum(traj_exp)